<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PostgreSQL_04D_Ex4_eCommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises


#Install & Configure Postgres

In [1]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Tested on  2024-01-07 10:06:56.631165+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


For Connecting to a Postgres server on Amazon RDS, see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PostgreSQL_01_Install_Config.ipynb)

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


#Create Schema, Tables and Insert Data

In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:upraxis:upass


In [5]:
# Original Data https://www.kaggle.com/carrie1/ecommerce-data
# Same data in Google Drive https://drive.google.com/open?id=1K_rQftggePnII3K6613MumhKo3dTMO5r
# 10% data in Google Drive https://drive.google.com/file/d/1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX/view?usp=sharing
#!wget -q -O eCommerce.csv 'https://drive.google.com/uc?export=download&id=1K_rQftggePnII3K6613MumhKo3dTMO5r'       # full data
#!wget -O eCommerce02.csv 'https://drive.google.com/uc?export=download&id=1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX'      # 10% data

!wget -q https://github.com/Praxis-QR/RDWH/raw/main/data/eCommerce.zip
!unzip eCommerce.zip
#!head eCommerce.csv

Archive:  eCommerce.zip
  inflating: eCommerce.csv           


In [4]:
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop schema if exists ec cascade'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'create schema if not exists ec'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dn+'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists ec.ecommerce'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS ec.ecommerce (\
invoiceno char(7), stockcode varchar(20), description varchar(50), quantity int, invoicedate date, unitprice decimal(10,2), customerid char(5), country varchar(20)) \
'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dt+ ec.*'
# --------------------------------------------------------------

NOTICE:  schema "ec" does not exist, skipping
DROP SCHEMA
CREATE SCHEMA
                          List of schemas
  Name  |  Owner   |  Access privileges   |      Description       
--------+----------+----------------------+------------------------
 ec     | upraxis  |                      | 
 public | postgres | postgres=UC/postgres+| standard public schema
        |          | =UC/postgres         | 
(2 rows)

NOTICE:  table "ecommerce" does not exist, skipping
DROP TABLE
CREATE TABLE
                                     List of relations
 Schema |   Name    | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+-----------+-------+---------+-------------+---------------+---------+-------------
 ec     | ecommerce | table | upraxis | permanent   | heap          | 0 bytes | 
(1 row)



In [41]:
# One way to handle UTF8 error
#
#!iconv -c -t utf8 eCommerce.csv > eCommerce.utf8.csv
#!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY ec.ecommerce FROM '/content/eCommerce.utf8.csv' DELIMITER ',' CSV HEADER;"

Password for user postgres: 
COPY 541909


In [6]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually, password = 'pass'
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY ec.ecommerce FROM '/content/eCommerce.csv' DELIMITER ',' CSV HEADER encoding 'windows-1251';"


Password for user postgres: 
COPY 541909


In [7]:
#!cat ItemsOrdered.csv
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'select * from ec.ecommerce limit 5'

 invoiceno | stockcode |             description             | quantity | invoicedate | unitprice | customerid |    country     
-----------+-----------+-------------------------------------+----------+-------------+-----------+------------+----------------
 536365    | 85123A    | WHITE HANGING HEART T-LIGHT HOLDER  |        6 | 2010-12-01  |      2.55 | 17850      | United Kingdom
 536365    | 71053     | WHITE METAL LANTERN                 |        6 | 2010-12-01  |      3.39 | 17850      | United Kingdom
 536365    | 84406B    | CREAM CUPID HEARTS COAT HANGER      |        8 | 2010-12-01  |      2.75 | 17850      | United Kingdom
 536365    | 84029G    | KNITTED UNION FLAG HOT WATER BOTTLE |        6 | 2010-12-01  |      3.39 | 17850      | United Kingdom
 536365    | 84029E    | RED WOOLLY HOTTIE WHITE HEART.      |        6 | 2010-12-01  |      3.39 | 17850      | United Kingdom
(5 rows)



In [9]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select max(invoicedate), min(invoicedate) from ec.ecommerce"

    max     |    min     
------------+------------
 2011-12-09 | 2010-12-01
(1 row)



## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [10]:
#!pip install psycopg2
import psycopg2
import pandas as pd

In [11]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [12]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [45]:
runSelect("select * from ec.ecommerce limit 10")

invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,2010-12-01,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01,1.85,17850,United Kingdom
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01,1.69,13047,United Kingdom


In [18]:
runSelect("select * from ec.ecommerce where description like '%WOODEN%' and invoicedate > '2011-01-01' limit 10")

invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
540526,22241,GARLAND WOODEN HAPPY EASTER,12,2011-01-09,1.25,14606,United Kingdom
540528,21380,WOODEN HAPPY BIRTHDAY GARLAND,2,2011-01-09,2.95,17800,United Kingdom
540534,22228,BUNNY WOODEN PAINTED WITH BIRD,12,2011-01-09,0.85,12971,United Kingdom
C540535,82494L,WOODEN FRAME ANTIQUE WHITE,-1,2011-01-09,2.95,15005,United Kingdom
540536,21892,TRADITIONAL WOODEN CATCH CUP GAME,1,2011-01-09,1.25,17757,United Kingdom
540537,22577,WOODEN HEART CHRISTMAS SCANDINAVIAN,6,2011-01-09,0.85,13565,United Kingdom
540538,82482,WOODEN PICTURE FRAME WHITE FINISH,1,2011-01-09,2.55,17841,United Kingdom
540543,82482,WOODEN PICTURE FRAME WHITE FINISH,3,2011-01-09,2.55,14395,United Kingdom
540543,82494L,WOODEN FRAME ANTIQUE WHITE,3,2011-01-09,2.95,14395,United Kingdom
540545,22577,WOODEN HEART CHRISTMAS SCANDINAVIAN,31,2011-01-09,0.85,13451,United Kingdom


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-12-31 16:01:25.642107+05:30
